In [5]:
import numpy 
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense ,Dropout, LSTM
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to C:\Users\Shweta
[nltk_data]     Patki\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
#load data
file=open("frankenstein.txt").read()

In [10]:
def tokenize_words(input):
    input=input.lower()
    tokenizer=RegexpTokenizer(r'\w+')
    tokens=tokenizer.tokenize(input)
    filtered=filter(lambda token:token not in stopwords.words('english'),tokens)
    return"".join(filtered)
 
processed_inputs=tokenize_words(file)

    

In [12]:
#chars to numbers
chars=sorted(list(set(processed_inputs)))
char_to_num=dict((c,i)for i ,c in enumerate(chars))

In [15]:
#check
input_len=len(processed_inputs)
vocab_len=len(chars)
print("Total number of characters:",input_len)
print("Total vocab:",vocab_len)

Total number of characters: 232972
Total vocab: 37


In [23]:
seq_length=100
x_data=[]
y_data=[]

In [25]:
for i in range (0,input_len-seq_length,1):
    in_seq=processed_inputs[i:i+seq_length]
    out_seq=processed_inputs[i+seq_length]
    x_data.append([char_to_num[char]for char in in_seq])
    y_data.append(char_to_num[out_seq])
n_patterns=len(x_data)
print("Total Patterns:",n_patterns)


Total Patterns: 232872


In [27]:
X=numpy.reshape(x_data,(n_patterns,seq_length,1))
X=X/float(vocab_len)

In [31]:
y=to_categorical(y_data)

In [32]:
#creating the model
model=Sequential()
model.add(LSTM(256,input_shape=(X.shape[1],X.shape[2]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1],activation='softmax'))

In [33]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [35]:
#saving weights
filepath="model_weights_saved.hdf5"
checkpoint=ModelCheckpoint(filepath,monitor='loss',verbose=1,save_best_only=True,mode='min')
desired_callbacks=[checkpoint]

In [36]:
model.fit(X,y, epochs=4,batch_size=256,callbacks=desired_callbacks)

Epoch 1/4

910/910 [==============================] - ETA: 0s - loss: 2.9273
Epoch 1: loss improved from inf to 2.92728, saving model to model_weights_saved.hdf5
910/910 [==============================] - 1147s 1s/step - loss: 2.9273
Epoch 2/4


C:\Users\Shweta Patki\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


910/910 [==============================] - ETA: 0s - loss: 2.9114
Epoch 2: loss improved from 2.92728 to 2.91140, saving model to model_weights_saved.hdf5
910/910 [==============================] - 1354s 1s/step - loss: 2.9114
Epoch 3/4
910/910 [==============================] - ETA: 0s - loss: 2.9000
Epoch 3: loss improved from 2.91140 to 2.89997, saving model to model_weights_saved.hdf5
910/910 [==============================] - 1155s 1s/step - loss: 2.9000
Epoch 4/4
910/910 [==============================] - ETA: 0s - loss: 2.8652
Epoch 4: loss improved from 2.89997 to 2.86524, saving model to model_weights_saved.hdf5
910/910 [==============================] - 1250s 1s/step - loss: 2.8652


In [37]:
filename="model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss="categorical_crossentropy",optimizer='adam')

In [38]:
num_to_char=dict((i,c)for i,c in enumerate(chars))

In [39]:
start=numpy.random.randint(0,len(x_data)-1)
pattern=x_data[start]
print("Random Seed:")
print("\"",''.join([num_to_char[value]for value in pattern]),"\"")

Random Seed:
" ionquitgenevaforevercountryhappybeloveddearadversitybecamehatefulprovidedsummoneytogetherjewelsbelon "


In [40]:
#generate the text
for i in range(1000):
    x=numpy.reshape(pattern,(1,len(pattern),1))
    x=x/float(vocab_len)
    prediction=model.predict(x,verbose=0)
    index=numpy.argmax(prediction)
    result=num_to_char[index]
    seq_in=[num_to_char[value]for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern=pattern[1:len(pattern)]

eneereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereereeree